# Linear Algebraic Equations

## Direct Methods
* Gauss elimination Method
* Gauss-Jordan Method
* LU Decomposition Methods
  * Dolittle LU Decomposition
  * Crout's Method
  * Cholesky Method
  
## Iterative Methods
* Gauss-Seidel Method
* Successuve Over-relaxation Method

## Gauss Elimination Method
Gauss uses a series of row operations to reduce the coefficient matrix to upper triangular form while also performing the same row operations on the right hand side vector. The equations are then solved by backward substitution.

Forward elimination consists of a series of row operation. A row operation is such that it does not affect the solution. Valid row operations are:

1. Interchange two rows
2. Multiply a row with a constant
3. Add a row multiplied with a constant to another row

Forward elimination begins by choosing the pivotal row, and performing a row operation on all rows below the pivotal row. If pivotal row is $p$, then all elements $a_{ip}, i=p+1 \text{ to } n$ are made zero using appropriate row operations, namely, $a_{ij} \leftarrow a_{ij} - a_{pj} \cdot \frac{a_{ip}}{a_{pp}}; \quad j = p, p+1, \ldots , n; \quad i = p+1, p+2, \ldots, n$

In [1]:
from __future__ import division, print_function
import numpy as np

def gauss_elim(a, b):
    '''Returns copy of coefficient matrix [a] and right hand side vector {b} after forward elimination along with
       solution vector {x}'''
    aa, bb = forward_elim(a, b)  # Forward elimination
    x = back_sub(a, b)           # Back substitution
    return aa, bb, x

def forward_elim(a, b):
    ra, ca = a.shape  # Determine size of coefficient matrix
    rb = b.shape[0]   # Determine size of right hand side vector
    if ra == rb:      # Check if number of rows in [a] matches number of elements in {b}
        for p in range(ra-1):  # Choose pivotal row
            for i in range(p+1, ra):  # Perform row operations on all rows below pivotal row
                f = a[i, p] / a[p, p] # Calculate multiplication factor
                a[i, p:] = f * a[p, p:] - a[i, p:]  # Row operation on row 'i' of [a]
                b[i] = f * b[p] - b[i]              #  Row operation on element 'i' of {b}
        return a, b
    else:
        print('Error')
        return None   # Return None if number of rows in [a] does not match number of elements in {b}

def back_sub(a, b):
    x = np.zeros(b.shape, dtype=float)
    x[-1] = b[-1] / a[-1, -1]  # Determine x_n
    for i in range(a.shape[1]-1, -1, -1):  # Determine remaining unknowns looping backwards
        x[i] = (b[i] - np.dot(a[i, i+1:], x[i+1:])) / a[i, i]
    return x

a = np.array([[10, 3, -2], [5, 9, -3], [-6, 8, 10]], dtype=float)
b = np.array([95, -10, 20], dtype=float)
print(a)
print(b)
aa, bb, x = gauss_elim(a, b)
print(aa)
print(bb)
print(x)

print(np.linalg.solve(a, b))

[[ 10.   3.  -2.]
 [  5.   9.  -3.]
 [ -6.   8.  10.]]
[ 95. -10.  20.]
[[ 10.           3.          -2.        ]
 [  0.          -7.5          2.        ]
 [  0.           0.          11.41333333]]
[  95.           57.5         152.13333333]
[ 13.39953271  -4.11214953  13.32943925]
[ 13.39953271  -4.11214953  13.32943925]


### Gauss Elimination with Partial Pivoting

In [13]:
def get_pivotal_row(a, r):
    n = a.shape[0]
    imax = r
    absmax = abs(a[r, r])
    for i in range(r+1, n):
        if abs(a[i, r] > absmax):
            imax = i
            absmax = abs(a[i, r])
    return imax

def forward_elim(a, b, pivoting=True):
    ra, ca = a.shape
    rb = b.shape[0]
    if ra == rb:
        for p in range(ra-1):
            if pivoting:  # If partial pivoting is to be performed
                imax = get_pivotal_row(a, p)  # Determine index of row containing largest absolute a_ip
                if imax != p:  # If current p is not the pivotal row, interchange rows p and imax
                    print('Interchange %d and %d' % (p, imax))
                    tmp = np.copy(a[p, p:])         # Use np.copy() to make a copy, otherwise it is a reference
                    a[p, p:] = np.copy(a[imax, p:])
                    a[imax, p:] = np.copy(tmp)
                    b[p], b[imax] = b[imax], b[p]
                    print(a)
                    print(b)
                    print()
            for i in range(p+1, ra):
                f = a[i, p] / a[p, p]
                a[i, p:] = f * a[p, p:] - a[i, p:]
                b[i] = f * b[p] - b[i]
        return a, b
    else:
        print('Error')

a = np.array([[10, 3, -2], [5, 9, -3], [-6, 8, 10]], dtype=float)
b = np.array([95, -10, 20], dtype=float)
print(a)
print(b)
aa, bb, x = gauss_elim(a, b)
print(aa)
print(bb)
print(x)

print(np.linalg.solve(a, b))
print()

a = np.array([[5, 9, -3], [10, 3, -2], [-6, 8, 10]], dtype=float)
b = np.array([-10, 95, 20], dtype=float)
print(a)
print(b)
aa, bb, x = gauss_elim(a, b)
print(aa)
print(bb)
print(x)

print(np.linalg.solve(a, b))

[[ 10.   3.  -2.]
 [  5.   9.  -3.]
 [ -6.   8.  10.]]
[ 95. -10.  20.]
[[ 10.           3.          -2.        ]
 [  0.          -7.5          2.        ]
 [  0.           0.          11.41333333]]
[  95.           57.5         152.13333333]
[ 13.39953271  -4.11214953  13.32943925]
[ 13.39953271  -4.11214953  13.32943925]

[[  5.   9.  -3.]
 [ 10.   3.  -2.]
 [ -6.   8.  10.]]
[-10.  95.  20.]
Interchange 0 and 1
[[ 10.   3.  -2.]
 [  5.   9.  -3.]
 [ -6.   8.  10.]]
[ 95. -10.  20.]

[[ 10.           3.          -2.        ]
 [  0.          -7.5          2.        ]
 [  0.           0.          11.41333333]]
[  95.           57.5         152.13333333]
[ 13.39953271  -4.11214953  13.32943925]
[ 13.39953271  -4.11214953  13.32943925]
